In [9]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import nltk
import pandas as pd
from ast import literal_eval
from collections import Counter
from scipy.optimize import fmin_l_bfgs_b
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import multiprocessing
import random
from tqdm import tqdm
import optimizeTopicVectors as ot
import time

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
# hoseong's class
from STMD import *
from ASUM import *
from ASUM_Embedding import *
from preprocess import *

In [3]:
# load data
# work_path = "/media/hs-ubuntu/data/dataset/MasterThesis/STMD_data/"
work_path = "E:/dataset/MasterThesis/STMD_data/"


In [3]:

data = pd.read_csv(work_path + "preprocess_complete_Electronics.csv")

In [4]:
def extract_sample(data, brand_name, count, ratio = 1, random_state = 42):
    brand = data[data['brand']==brand_name]
    pos_reviews = brand[brand.overall >= 4]
    neg_reviews = brand[brand.overall <= 2]
    if ratio == 1:
        pos_sample = pos_reviews.sample(count, random_state=random_state)
        neg_sample = neg_reviews.sample(count, random_state=random_state)
        df = pd.concat([pos_sample, neg_sample], axis=0)
    else:
        df = brand.sample(count * 2, random_state = random_state)
    df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
    return df

In [131]:
# Apple 리뷰에서 긍정, 부정인 문서만
apple = data[data.brand == "Apple"]
pos_reviews = apple[apple.overall >= 4]
neg_reviews = apple[apple.overall <= 2]
df = pd.concat([pos_reviews, neg_reviews], axis=0)
df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
df.reset_index(drop=True, inplace=True)

In [129]:
# #canon
# canon = data[data.brand == "Canon"]
# pos_reviews = canon[canon.overall >= 4]
# neg_reviews = canon[canon.overall <= 2]
# df = pd.concat([pos_reviews, neg_reviews], axis=0)
# df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
# df.reset_index(drop=True, inplace=True)

In [3]:
# df.to_csv(work_path + "apple_pos_neg.csv", index=False)
# print(df[df.overall >= 4].shape)
# print(df[df.overall <= 2].shape)

NameError: name 'df' is not defined

In [4]:
df = pd.read_csv(work_path + "apple_pos_neg.csv")
df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))

In [5]:
# prepare
sentence_list, review_label, numSentence = prepare(df, max_sentence = 50)
documents, sentence_list_again, bigram, documents_label\
= bigram_and_sentence(sentence_list, review_label, numSentence, max_vocab=3000, threshold = 10, min_count = 10)

C:\Program Files\Anaconda3\lib\site-packages\gensim\models\phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [123]:
asum_embedding = ASUM_Embedding(review_label, wordVectors, sentimentVector, numTopics=30, alpha=0.01, beta=0.001, gamma=1, binary=0.8, numSentiments=2)
asum_embedding._initialize_(sentence_list_again)

In [124]:
#Save WordList
save_path = "E:/dataset/MasterThesis/STMD_data/canon_pos_neg_data_txt/"
from collections import OrderedDict
dic = OrderedDict(sorted(asum_embedding.word2idx.items(), key=lambda t: t[1]))
word_list = []
for key in dic.keys():
    word_list.append(key)
with open(save_path + "WordList.txt", 'w') as f:
    for word in word_list:
        f.writelines(word + "\n")

In [125]:
#Save BagOfSentence
with open(save_path + "BagOfSentences.txt", 'w') as f:
    for doc in range(asum_embedding.numDocs):
        f.write(str(len(asum_embedding.doc_sent_word_dict[doc])))
        f.write("\n")
        for sent in asum_embedding.doc_sent_word_dict[doc]:
            for word in sent:
                f.write(str(word) + ' ')
            f.write("\n")

In [126]:
#Save document pos_neg label
with open(save_path + "ReviewLabel.txt", 'w') as f:
    for label in asum_embedding.review_label:
        f.writelines(str(label) + "\n")

In [6]:
# word / sentiment embedding

window = [2]
size = [100]
passes = 10
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=8, batch_words = 10000,
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
#             if (epoch + 1) % 5 ==0:
#                 model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:57<00:00,  5.69s/it]


In [7]:
model.most_similar([model.docvecs["positive"]], topn=20)

[('retina_display', 0.9619499444961548),
 ('full_size', 0.9572002291679382),
 ('larg', 0.9389293789863586),
 ('mac', 0.9361025094985962),
 ('travel', 0.9357125759124756),
 ('long_time', 0.9325348734855652),
 ('portabl', 0.9324036240577698),
 ('read_book', 0.9319947361946106),
 ('video', 0.9307326078414917),
 ('hd', 0.9302165508270264),
 ('color', 0.9299668669700623),
 ('smooth', 0.9298363924026489),
 ('much_better', 0.9295869469642639),
 ('watch_movi', 0.9290027618408203),
 ('youtub', 0.9285895228385925),
 ('excel', 0.9278790950775146),
 ('far', 0.926765501499176),
 ('carri_around', 0.9265998005867004),
 ('photo', 0.9201708436012268),
 ('kindl', 0.9195323586463928)]

In [8]:
model.most_similar([model.docvecs["negative"]], topn=20)

[('certifi', 0.9686375260353088),
 ('warranti', 0.9640694856643677),
 ('defect', 0.9583691954612732),
 ('item', 0.9580919146537781),
 ('seller', 0.9580525755882263),
 ('sent_back', 0.9571664929389954),
 ('call', 0.9553946852684021),
 ('return', 0.9533596634864807),
 ('amazon', 0.9518277049064636),
 ('ha', 0.9513228535652161),
 ('repair', 0.9509011507034302),
 ('broke', 0.9484753608703613),
 ('serial_number', 0.9483621716499329),
 ('fix', 0.9459980130195618),
 ('refund', 0.9414042234420776),
 ('playlist', 0.9408608675003052),
 ('exchang', 0.9374716281890869),
 ('told', 0.9370841979980469),
 ('fault', 0.935389518737793),
 ('receiv', 0.933317244052887)]

### test 1

In [6]:
#model.save("E:/dataset/MasterThesis/gensim_models/test1")
model = Doc2Vec.load("E:/dataset/MasterThesis/gensim_models/test1")

In [7]:
wordVectors = np.zeros((len(model.index2word), model.vector_size))
for index, word in enumerate(model.index2word):
    wordVectors[index,:] = model[word]

sentimentVector = np.zeros((2, model.vector_size))
sentimentVector[0,:] = model.docvecs['positive']
sentimentVector[1,:] = model.docvecs['negative']

In [20]:
%%time
for binary in [0,0.2,0.4,0.6,0.8,1]:
    asum_embedding_path = "D:/Dropbox/2016-2/졸업논문/RESULT/STMD_result/"
    asum_embedding = ASUM_Embedding(review_label, wordVectors, sentimentVector, numTopics=30, alpha=0.1, beta=0.001, gamma=1, binary=binary, numSentiments=2)
    asum_embedding._initialize_(sentence_list_again)
    asum_embedding.run(save_path=asum_embedding_path, print_iter=10, maxIters= 100)

iteration 1, time 27.3832
iteration 2, time 27.3219
iteration 3, time 27.7294
iteration 4, time 27.2232
iteration 5, time 27.3976
iteration 6, time 27.2669
iteration 7, time 27.0035
iteration 8, time 27.2442
iteration 9, time 27.4283
Starting iteration 10 of 100:
0.739581083999
iteration 10, time 27.1601
iteration 11, time 27.3351
iteration 12, time 27.3032
iteration 13, time 27.1149
iteration 14, time 27.3900
iteration 15, time 27.0653
iteration 16, time 27.2071
iteration 17, time 27.6749
iteration 18, time 27.3653
iteration 19, time 27.1090
Starting iteration 20 of 100:
0.730079896351
iteration 20, time 27.3686
iteration 21, time 27.1111
iteration 22, time 27.2117
iteration 23, time 27.3357
iteration 24, time 27.2561
iteration 25, time 27.0165
iteration 26, time 27.3173
iteration 27, time 27.5331
iteration 28, time 27.3727
iteration 29, time 27.1113
Starting iteration 30 of 100:
0.725545238609
iteration 30, time 27.3333
iteration 31, time 27.2756
iteration 32, time 27.0623
iteration 

In [21]:
%%time
asum_embedding_path = "D:/Dropbox/2016-2/졸업논문/RESULT/STMD_result/"
asum_embedding = ASUM_Embedding(review_label, wordVectors, sentimentVector, numTopics=30, alpha=0.1, beta=0.001, gamma=1, binary=0.5, numSentiments=2)
asum_embedding._initialize_(sentence_list_again)
asum_embedding.run(save_path=asum_embedding_path, print_iter=100, maxIters= 1000)

iteration 1, time 27.0980
iteration 2, time 27.5377
iteration 3, time 27.5136
iteration 4, time 27.5445
iteration 5, time 27.2968
iteration 6, time 27.4674
iteration 7, time 27.6237
iteration 8, time 27.2423
iteration 9, time 27.4042
iteration 10, time 27.4038
iteration 11, time 27.3359
iteration 12, time 27.1377
iteration 13, time 27.3850
iteration 14, time 27.4886
iteration 15, time 27.2723
iteration 16, time 27.5257
iteration 17, time 27.2282
iteration 18, time 27.4465
iteration 19, time 27.3702
iteration 20, time 27.3970
iteration 21, time 27.4385
iteration 22, time 27.3986
iteration 23, time 27.3240
iteration 24, time 27.4849
iteration 25, time 27.0661
iteration 26, time 27.2586
iteration 27, time 27.1930
iteration 28, time 27.2405
iteration 29, time 27.4195
iteration 30, time 27.6327
iteration 31, time 27.3718
iteration 32, time 27.2119
iteration 33, time 27.2607
iteration 34, time 27.6304
iteration 35, time 27.1681
iteration 36, time 27.2461
iteration 37, time 27.4907
iteration 

In [19]:
# topic = 30
# binaries = [0, 0.2, 0.4, 0.6, 0.8, 1]
# for binary in binaries:
#     print("ratio is ", binary)
#     asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding_topic_" + str(topic)
#     asum_embedding = ASUM_Embedding(review_label, wordVectors, sentimentVector, numTopics=topic, alpha=0.1, beta=0.001, gamma=1, binary=binary, numSentiments=2)
#     asum_embedding._initialize_(sentence_list_again)
#     asum_embedding.run(save_path=asum_embedding_path, print_iter=5, save_iter = 50, maxIters= 50)

ratio is  0
iteration 1, time 28
iteration 2, time 28
iteration 3, time 28
iteration 4, time 29
Starting iteration 5 of 50:
0.75145756856
iteration 5, time 28
iteration 6, time 29
iteration 7, time 28
iteration 8, time 28
iteration 9, time 28
Starting iteration 10 of 50:
0.731591448931
iteration 10, time 28
iteration 11, time 28
iteration 12, time 28
iteration 13, time 28
iteration 14, time 28
Starting iteration 15 of 50:
0.735478298424
iteration 15, time 28
iteration 16, time 28
iteration 17, time 28
iteration 18, time 28
iteration 19, time 28
Starting iteration 20 of 50:
0.725977110775
iteration 20, time 28
iteration 21, time 28
iteration 22, time 28
iteration 23, time 28
iteration 24, time 28
Starting iteration 25 of 50:
0.724897430361
iteration 25, time 28
iteration 26, time 28
iteration 27, time 28
iteration 28, time 28
iteration 29, time 28
Starting iteration 30 of 50:
0.722738069531
iteration 30, time 28
iteration 31, time 28
iteration 32, time 28
iteration 33, time 28
iteration

In [15]:
df_result

,T0-S0,T1-S0,T2-S0,T3-S0,T4-S0,T5-S0,T6-S0,T7-S0,T8-S0,T9-S0,...,T20-S1,T21-S1,T22-S1,T23-S1,T24-S1,T25-S1,T26-S1,T27-S1,T28-S1,T29-S1
0,"(use, 0.407626684397)","(use, 0.390307102333)","(use, 0.344009064471)","(use, 0.33761840215)","(use, 0.34098537136)","(use, 0.330183540191)","(use, 0.395928973838)","(use, 0.372225066803)","(use, 0.332120174538)","(ipad, 0.324868282113)",...,"(appl, 0.409684376317)","(appl, 0.465965972778)","(appl, 0.397938631798)","(appl, 0.497572034502)","(appl, 0.350278360164)","(appl, 0.434667998214)","(appl, 0.442312743271)","(appl, 0.407682401658)","(appl, 0.403302114862)","(appl, 0.411840870769)"
1,"(ipad, 0.234577620266)","(ipad, 0.225765872918)","(ipad, 0.274277497348)","(time_capsul, 0.243205211756)","(ipad, 0.269378443374)","(ipad, 0.209384684024)","(ipad, 0.223236549079)","(ipad, 0.253789818275)","(ipad, 0.245136319302)","(use, 0.310535857902)",...,"(product, 0.323936483599)","(product, 0.244749803884)","(product, 0.275118066428)","(product, 0.309170972894)","(product, 0.239430777834)","(product, 0.270961869017)","(one, 0.223820906234)","(product, 0.222866379573)","(one, 0.252063821789)","(one, 0.287204817773)"
2,"(devic, 0.180740133648)","(devic, 0.187500470729)","(veri, 0.199897159084)","(ipad, 0.185879794442)","(work, 0.173902539393)","(appl, 0.205358055485)","(appl, 0.138996341879)","(work, 0.164963381878)","(one, 0.173967710472)","(work, 0.181544040004)",...,"(one, 0.185787100888)","(would, 0.197682533906)","(one, 0.221077017666)","(one, 0.212555043865)","(one, 0.23499687454)","(one, 0.231446596452)","(product, 0.202504629449)","(one, 0.173944491374)","(product, 0.229658148741)","(product, 0.233015229514)"
3,"(work, 0.169203529373)","(work, 0.172194309853)","(love, 0.176653303377)","(get, 0.174499398864)","(screen, 0.167082831966)","(work, 0.140931998862)","(comput, 0.138996341879)","(great, 0.152273890965)","(great, 0.173967710472)","(great, 0.167211615793)",...,"(ipad, 0.171495785435)","(one, 0.18826907991)","(get, 0.171948791518)","(would, 0.159416282899)","(use, 0.181790035022)","(would, 0.169351168135)","(get, 0.175859283469)","(like, 0.135894133886)","(get, 0.184846802645)","(get, 0.151730847126)"
4,"(appl, 0.157666925097)","(appl, 0.130102367444)","(appl, 0.167355761094)","(great, 0.144151677323)","(appl, 0.146623709685)","(one, 0.128852113245)","(appl_tv, 0.132730590482)","(one, 0.143814230356)","(appl, 0.170013898871)","(like, 0.157656666319)",...,"(get, 0.1619682418)","(get, 0.169442171919)","(amazon, 0.167527751628)","(get, 0.144923893544)","(would, 0.164054421849)","(use, 0.163706129197)","(would, 0.149213937489)","(ipod, 0.130458368531)","(bought, 0.140035456549)","(purchas, 0.1463118883)"
5,"(great, 0.146130320822)","(veri, 0.122449287006)","(great, 0.167355761094)","(drive, 0.141059022818)","(app, 0.146623709685)","(like, 0.128852113245)","(app, 0.130572321159)","(devic, 0.135354569746)","(app, 0.146291029261)","(love, 0.138546767372)",...,"(amazon, 0.14767736751)","(ipad, 0.145908536931)","(ipad, 0.142471855829)","(use, 0.13043150419)","(get, 0.159620518556)","(get, 0.15806109026)","(return, 0.141399841286)","(problem, 0.116037910009)","(amazon, 0.138915587886)","(ipad, 0.140892929474)"
6,"(app, 0.138439251305)","(great, 0.118622746787)","(comput, 0.134814363103)","(pc, 0.13772542694)","(great, 0.143213855971)","(comput, 0.120798856167)","(get, 0.12214830044)","(love, 0.135354569746)","(devic, 0.142337217659)","(veri, 0.138546767372)",...,"(buy, 0.147676926347)","(time, 0.136495082935)","(new, 0.142471855829)","(ipad, 0.13043150419)","(new, 0.159620518556)","(time, 0.152416051322)","(like, 0.133226729901)","(ipad, 0.114151072464)","(use, 0.134434038287)","(ipod, 0.130055011822)"
7,"(like, 0.134593716546)","(one, 0.114796206569)","(get, 0.116219278537)","(comput, 0.132771281745)","(one, 0.136394148544)","(get, 0.11274559909)","(great, 0.11793629008)","(app, 0.135354569746)","(screen, 0.126521971253)","(screen, 0.119436868424)",...,"(would, 0.138149382712)","(purchas, 0.1270816

In [116]:
# 토픽별 단어 추출
wkl = asum_embedding.n_wkl

In [117]:
df = pd.DataFrame()
count = np.zeros((asum_embedding.numTopics*asum_embedding.numSentiments,asum_embedding.vocabSize))
col_name = []
idx = 0
for l in range(asum_embedding.numSentiments):
    for t in range(asum_embedding.numTopics):
        name = 'T' + str(t) + '-' + 'S' + str(l)
        count[idx] = wkl[:,t,l]
        idx+=1
        col_name.append(name)

In [103]:
count.shape

(60, 5000)

In [104]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(count)
score_result = tfidf.toarray()

In [15]:
len(asum_embedding.idx2word.keys())

3000

In [105]:
word_list = [asum_embedding.idx2word[i] for i in range(5000)]
df_result = pd.DataFrame()

In [106]:
for i in range(60):
    scores = score_result[i]
    word_score_list = []
    for word, score in zip(word_list, scores):
        word_score = tuple((word,score))
        word_score_list.append(word_score)
    word_score_list = sorted(word_score_list, key=lambda x:-x[1])
    df_result[col_name[i]] = word_score_list

In [107]:
df_result

,T0-S0,T1-S0,T2-S0,T3-S0,T4-S0,T5-S0,T6-S0,T7-S0,T8-S0,T9-S0,...,T20-S1,T21-S1,T22-S1,T23-S1,T24-S1,T25-S1,T26-S1,T27-S1,T28-S1,T29-S1
0,"(vs, 0.311741792404)","(app, 0.426597836172)","(tv, 0.406183604628)","(camera, 0.281911236276)","(time_capsul, 0.443926374587)","(de, 0.502164023217)","(great, 0.448143865372)","(light, 0.431567755281)","(work, 0.305719567787)","(sleep, 0.320449124018)",...,"(x, 0.585436770753)","(product, 0.435396771895)","(anyon, 0.345303987259)","(ever, 0.377758532358)","(product, 0.259288328761)","(cabl, 0.48184481946)","(ipad, 0.381468568187)","(gift, 0.367566138758)","(appl, 0.311854250397)","(order, 0.296858364817)"
1,"(faster, 0.203849270183)","(remot, 0.265731931372)","(appl_tv, 0.33809735885)","(return, 0.241171001624)","(pc, 0.328112788701)","(la, 0.42287496692)","(product, 0.300878140448)","(great, 0.278099007492)","(great, 0.305719567787)","(timer, 0.275399662793)",...,"(camera, 0.254425015635)","(appl, 0.392020735139)","(would, 0.281144888734)","(best, 0.353470810081)","(friend, 0.190179353931)","(charger, 0.297196258597)","(overal, 0.230709335009)","(bought, 0.311227451892)","(one, 0.212278565499)","(cabl, 0.263399858819)"
2,"(gen, 0.191297068942)","(appl_tv, 0.235364323405)","(itun, 0.249011770944)","(great, 0.228628674802)","(drive, 0.31042499926)","(que, 0.370124076005)","(bad, 0.219083558727)","(design, 0.245380266065)","(music, 0.291777588895)","(minut, 0.266655273588)",...,"(ram, 0.252803219624)","(great, 0.287686349208)","(product, 0.267225589028)","(expect, 0.333367348794)","(creat, 0.181157246119)","(appl, 0.229724013017)","(ever, 0.216503201239)","(one, 0.309762747331)","(would, 0.163457404568)","(cover, 0.242563024625)"
3,"(chip, 0.163251370988)","(cabl, 0.217260909005)","(stream, 0.237303041803)","(price, 0.216382259345)","(speed, 0.275303716862)","(muy, 0.250786165749)","(job, 0.201393650844)","(thin, 0.215090948913)","(thank, 0.267980923654)","(inform, 0.200280702511)",...,"(mp, 0.202632728854)","(anoth, 0.220678029262)","(amazon, 0.233499845024)","(purchas, 0.269117753653)","(fulfil_amazon, 0.179828422773)","(origin, 0.204216489649)","(appl, 0.203898464422)","(ipod, 0.253896079175)","(product, 0.145536309214)","(not, 0.2395477941)"
4,"(retina, 0.159414224119)","(tv, 0.21536355845)","(movi, 0.201100706213)","(money, 0.196144382396)","(mbp, 0.253279419513)","(el, 0.227768662157)","(love, 0.198589710573)","(look, 0.210959273097)","(love, 0.238695896719)","(cool, 0.192759561174)",...,"(slot, 0.190492784817)","(brand, 0.204067164361)","(highli_recommend, 0.230202658173)","(product, 0.241421066414)","(appl, 0.179107181831)","(not_work, 0.181776647998)","(say, 0.186280708687)","(son, 0.233809315589)","(get, 0.144677540227)","(match, 0.191915401352)"
5,"(drive, 0.158540517185)","(lot, 0.175606494912)","(airplay, 0.180944196444)","(also, 0.195518928464)","(file, 0.190517103116)","(lo, 0.185007798027)","(excel, 0.182417077428)","(size, 0.179983567793)","(pleasur, 0.232831761476)","(return, 0.174751006994)",...,"(gp, 0.190492784817)","(macbook, 0.170055970301)","(appl, 0.200997956148)","(one, 0.216380069911)","(charg, 0.1746687918)","(month, 0.181525768577)","(best, 0.164639108792)","(replac, 0.215444083291)","(buy, 0.13440024517)","(not_know, 0.186314950967)"
6,"(free, 0.152886952637)","(stream, 0.16936023232)","(watch, 0.166007847296)","(well_worth, 0.190789142371)","(transfer, 0.183955555117)","(se, 0.170826496618)","(hope_help, 0.18210545317)","(well, 0.161890400274)","(realli, 0.181658244748)","(satisfact, 0.17043553781)",...,"(ye, 0.190217673302)","(friend, 0.154168498177)","(buy, 0.170405518971)","(exactli, 0.194817544336)","(say, 0.173256888488)","(charg, 0.177141032393)","(product, 0.161645071874)","(christma, 0.181751118597)","(problem, 0.132396202947)","(back, 0.173897903406)"
7,"(screen, 0.14261675225)","(great, 0.153416049529)","(netflix, 0.162318784023)","(deal, 0.186905996064)","(share, 0.172458332922)","(en, 0.158578112595)","(hate, 0.164615194948)","(sleek, 0.161869835949)

In [61]:
for i in range(60):
    scores = score_result[i]
    df_result[col_name[i]] = score_result[i]

In [63]:
df_result.to_csv(r"D:/Dropbox/2016-2/졸업논문/RESULT/wordList/test1.csv")

In [ ]:
#count와 words_list를 합치기
